# <center>Missão Padrão Delta</center>

## <center><u>``Resumo do código``</u></center>

>### Entra: 
> - 1_Ninjas
> - 2_Delta  
>### Sai: 
> - Long_DataFusion
> - Wide_DataFusion
> - Wide_DataFusion_2022
> - G_Classic
> - G_Classic_Reposto
> - G_Classic_SemReposto
> - G_Rotura
> - G_NumProdutos
---
O objectivo é receber dados dos Ninjas e da Delta e devolver ficheiros prontos para fazer os Dashboards. 


---


In [387]:
import pandas as pd
import numpy as np
import datetime
import json
from ipywidgets import interact

def escrever_excel(dfa, nome):
    dfa.to_excel(f'D:\\B&N Dados\\Delta\\JM\\Padrao\\Padrao_08_2023\\{nome}.xlsx', index=False)
    
def escrever_csv(dfa, nome):
    dfa.to_csv(f'D:\\B&N Dados\\Delta\\JM\\Padrao\\Padrao_08_2023\\{nome}.csv', index=False)
    
def escrever_txt(dfa, nome):
    dfa.to_csv(f'D:\\B&N Dados\\Delta\\JM\\Padrao\\Padrao_08_2023\\{nome}.txt', sep='\t', index=False, header=False)
    
def ler_json(ficheiro):
    with open(f'D:\\B&N Dados\\Delta\\{ficheiro}.json', 'r') as file:
        mapa = json.load(file)
    return mapa

<a id="section0"></a>

# Índice

1. [Data Cleaning](#section1)<br>
2. [DataFusion Long](#section2)<br>
3. [DataFusion Wide](#section3)<br>
4. [Gráficos](#section4)<br>
5. [Escrever](#section5)<br>
6. [Visualizar](#section6)

# O que é preciso substituir:

1. Data Cleaning
   1. [Directória e nº bases Ninjas](#section1.1)
   2. [Directória e nº bases Delta](#section1.2)


---

<a id="section1"></a>
# <center>1. Data Cleaning</center>
[Voltar ao índice](#section0)<br>


In [388]:
''' Ler os dicionários previamente criados para poder usar os códigos de produto e de loja em vez dos nomes '''

mapa_produtos = ler_json("dicionário_produtos")
mapa_produtos = {int(key): value for key, value in mapa_produtos.items()}

mapa_lojas = ler_json("dicionário_lojas")
mapa_lojas = {int(key): value for key, value in mapa_lojas.items()}

mapa_produtos2 = ler_json("dicionário_produtos_23")
mapa_produtos2 = {int(key): value for key, value in mapa_produtos2.items()}   

- # `Ninjas`

> <font color=blue>Ler</font>
<a id="section1.1"></a>

In [389]:
%%time
# Ler o ficheiro long com Stocks e Fornecimento

directória = "D:\\B&N Dados\\Delta\\JM\\Padrao\\Padrao_08_2023\\"
num_bases = 1

prefixo = "1_Ninjas"
extensão = ".xlsx" #xlsb


dataframes = []

for i in range(1, num_bases + 1):
    file_path = f"{directória}{prefixo}{i}{extensão}"
    df = pd.read_excel(file_path)
    dataframes.append(df)
    
dfNinjas = pd.concat(dataframes, ignore_index=True)
dfNinjas = dfNinjas.rename(columns={' Data de Resposta': 'DATA', "Código da loja":"STORE", "Nome da Loja":"Loja"})

#dfNinjas = dfNinjas.rename(columns=lambda x: mapa_produtos2.get(x, x))

"""Datetime"""

# se xlsx
dfNinjas['DATA'] = pd.to_datetime(dfNinjas['DATA'], format='%d-%m-%Y') 

# se xlsb
#dfNinjas['DATA'] = pd.to_datetime(dfNinjas['DATA'], unit='d', origin=datetime.datetime(1899, 12, 30)) 

CPU times: total: 78.1 ms
Wall time: 77.1 ms


- Lista de produtos e lista de lojas

# <font color=red>É preciso definir melhor o formato do ficheiro dos ninjas para se saber os produtos, preciso de perceber melhor como é gerado na aplicação</font>

In [390]:
# Produtos
ficheiro_Produtos = dfNinjas.columns.tolist()
df_produtos = pd.DataFrame(ficheiro_Produtos[3:], columns=["DESC_ARTIGO"]) # Se não houver hora
#df_produtos = pd.DataFrame(ficheiro_Produtos[4:], columns=["DESC_ARTIGO"]) # Se houver hora

# Lojas
df_lojas = pd.DataFrame(dfNinjas["STORE"].unique().tolist(), columns=["STORE"])


'''Colocar em listas para serem chamáveis'''

lojas = df_lojas["STORE"].tolist()
produtos = df_produtos["DESC_ARTIGO"].tolist()
numprod = len(produtos)

In [391]:
produtos

['CERVEJA FRANZISKANER WEISSBIER 50 CL 5%',
 'CERVEJA CORONA 35,5CL (UN)',
 'CERVEJA LEFFE BLONDE 33 CL CX24']

[Escrever](#section5)

---

<a id="section2.1"></a>
- #  `Delta`

> <font color=blue>Ler</font>
<a id="section1.2"></a>

In [392]:
%%time
# Ler o ficheiro long com Stocks e Fornecimento

directória = "D:\\B&N Dados\\Delta\\JM\\Padrao\\Padrao_08_2023\\"
#extensão = ".xlsb"
extensão = ".xlsx"
num_bases = 1

prefixo = "2_Delta"

dataframes = []

for i in range(1, num_bases + 1):
    file_path = f"{directória}{prefixo}1{extensão}"
    df = pd.read_excel(file_path, sheet_name=f"{i}")
    dataframes.append(df)
    
dfDelta=pd.concat(dataframes, ignore_index=True)


"""Datetime"""

# se xlsx
dfDelta['DATA'] = pd.to_datetime(dfDelta['DATA'], format='%d-%m-%Y') 

# se xlsb
#dfDelta['DATA'] = pd.to_datetime(dfDelta['DATA'], unit='d', origin=datetime.datetime(1899, 12, 30)) 

CPU times: total: 547 ms
Wall time: 769 ms


> - **Definir os produtos e lojas em causa**

In [393]:
# Mudar nomes
# dfDelta = dfDelta.rename(columns={"Produto": "DESC_ARTIGO", "Loja Codigo": "STORE", "Quantidade Sellout": "SELLOUT_1_Dias_Antes", "Stock Sellout": "STOCK_1_Dias_Antes"})
dfDelta = dfDelta.rename(columns={"Produto": "DESC_ARTIGO","Loja Descricao":"Loja", "Loja Codigo": "STORE", "Quantidade Sellout": "SELLOUT", "Stock Sellout": "STOCK"})

# Estabelecer o Dataframe mais limitado
dfDelta_Limpo = dfDelta[dfDelta['STORE'].isin(lojas) & dfDelta['DESC_ARTIGO'].isin(produtos)].copy()



> - **Criar colunas de Stock e Sellout para o próprio dia**

In [394]:
# Organizar para atingir os valores relevantes
dfDelta_Limpo = dfDelta_Limpo.sort_values(by=["STORE", "DESC_ARTIGO", "DATA"])

#Agrupar por loja e produto para não haver valores errados 
# dfDelta_Limpo["SELLOUT"] = dfDelta_Limpo.groupby(["STORE", "DESC_ARTIGO"])["SELLOUT_1_Dias_Antes"].shift(-1)
# dfDelta_Limpo["STOCK"] = dfDelta_Limpo.groupby(["STORE", "DESC_ARTIGO"])["STOCK_1_Dias_Antes"].shift(-1)

> - **Retirar os dias que não têm interesse (último dia)**


### <center><font color=green>   Ainda fazer

> - **Id**

> - **Número de produtos presentes**

In [395]:
# Criar coluna com o número dos produtos em causa presentes no instante
dfNinjas['CERVEJA FRANZISKANER WEISSBIER 50 CL 5%'] = pd.to_numeric(dfNinjas['CERVEJA FRANZISKANER WEISSBIER 50 CL 5%'], errors='coerce') 
dfNinjas["Num_Produtos"] = dfNinjas[produtos].sum(axis=1)

dfNinjas.loc[(dfNinjas["Num_Produtos"] == 0) & (dfNinjas[produtos[1]].isna()), "Num_Produtos"] = np.nan

> - **Vendedor**

In [396]:
# Ler ficheiro
dfVendedor=pd.read_excel("D:\\B&N Dados\\Delta\\JM\\Padrao\\Padrao_08_2023\\2_Delta1.xlsx", sheet_name = "3")
dfVendedor = dfVendedor.rename(columns={"Loja Codigo":"STORE"})
dfVendedor = dfVendedor.drop_duplicates(subset=["STORE","Vendedor"])
# Criar coluna de reposição
dfNinjas = pd.merge(dfNinjas, dfVendedor[["STORE","Vendedor"]], how="left", on = "STORE")

> - **Reposição**

## <center><font color=orange>Output DataCleaning

In [397]:
[f"{index}: {column}" for index, column in enumerate(dfDelta_Limpo)]

['0: Vendedor',
 '1: STORE',
 '2: Loja',
 '3: DATA',
 '4: EAN',
 '5: DESC_ARTIGO',
 '6: STOCK',
 '7: SELLOUT']

In [398]:
#dfNinjas = dfNinjas.iloc[:, np.r_[5+numprod,0:5+numprod,6+numprod]].copy() # Se não houver hora
#dfNinjas1 = dfNinjas.iloc[:, np.r_[6+numprod,0:6+numprod,7+numprod]].copy() # Se houver hora
dfDelta = dfDelta_Limpo.copy()

---

<a id="section2"></a>
# <center> 2. DataFusion Long (1)</center>
[Voltar ao índice](#section0)

> - __dfNinjas__ já definido
> - __dfDelta__ já definido
> - __produtos__ já definidos

- #  `dfFinal1`

In [399]:
resto = dfNinjas.columns.difference(produtos)
# Passar ficheiro ninjas para long
dfNinjasLong = dfNinjas.melt(id_vars=resto, value_vars=produtos, var_name='DESC_ARTIGO', value_name='NinjaInfo')


#Mergir
dfMeio = pd.merge(dfNinjasLong, dfDelta, how="left", on = ["DATA","STORE", "DESC_ARTIGO"]) 
#dfMeio = dfMeio.drop(columns=["Loja"])

dfFinal1=dfMeio.copy()

> STK

> Balanço

In [400]:
# Definir coluna de Balanço (razão entre sellout e stock total)

dfFinal1["Balance"] = dfFinal1["SELLOUT"] / dfFinal1["STOCK"]

> Rotura

In [401]:
# Definir coluna de rotura (se stock menor ou igual a 0 e existe Linear)

dfFinal1["ROTURA"] = np.where((dfFinal1["STOCK"] <= 0), 1, 0)

>Sinal

In [402]:
# Categorias possíveis
mapping = {1: "Presente com Stock",
           2: "Ausente com Stock",
           3: "Presente sem Stock",
           4: "Ausente sem Stock"}
#            5: "Presente sem Registo",
#            6: "Ausente sem Registo"}

# Definir coluna de sinal
dfFinal1["Sinal"] = np.where((dfFinal1["STOCK"] > 0) & (dfFinal1["NinjaInfo"] == 1), 1,
                     np.where((dfFinal1["STOCK"] > 0) & (dfFinal1["NinjaInfo"] == 0), 2,
                     np.where((dfFinal1["STOCK"] <= 0) & (dfFinal1["NinjaInfo"] == 1), 3,
                     np.where((dfFinal1["STOCK"] <= 0) & (dfFinal1["NinjaInfo"] == 0), 4,
                     np.where((dfFinal1["STOCK"].isna()), np.nan, np.nan)))))

# Substituir números pelas expressões escolhidas
dfFinal1["Sinal"] = dfFinal1["Sinal"].map(mapping)
dfFinal1["NinjaInfo"] = np.where(dfFinal1["STOCK"].isna(), np.nan, dfFinal1["NinjaInfo"])

> Ciclos

> Adequação

> Dias para a rotura de stock

In [403]:
dfFinal1["Dias_para_Rotura"] = dfFinal1["STOCK"] / dfFinal1["SELLOUT"]

> Dias para a rotura de prateleira

> Roturas de prateleira (meh)

- ###  `Organizar`

## <center><font color=orange>Output DataFusion Long

In [404]:
df1 = dfFinal1
#df1 = dfFinal1.iloc[:, np.r_[5,0:2,7:16,2:5,6,16:19,19:24]].copy()

[Escrever](#section5) <br>
[Visualizar](#section6)

<a id="section3"></a>
# <center>3. DataFusion Wide (2)</center>
[Voltar ao índice](#section0)<br>

> - __dfNinjas__ já definido
> - __dfDelta__ já definido
> - __produtos__ já definidos

- #  `dfFinal2`

In [405]:
# Dataframe mergido no fim
dfMeio2 = dfNinjas.copy()


for coluna in produtos:       
    
    #Cada dfStocks é um dataframe com o produto específico
    dfStocks=dfDelta[dfDelta["DESC_ARTIGO"]==coluna][["DATA", "STORE", "STOCK", "SELLOUT"]]   #Dados Ninjas, seleccionar colunas do café específico
 
    #Juntar dados de acordo com o dia e a loja
    dfMeio2=pd.merge(dfMeio2, dfStocks, how="left", on = ["DATA","STORE"])  
    
    #Mudar nomes para ser adaptado a cada produto
    dfMeio2=dfMeio2.rename(columns={"STOCK": "STOCK %s" % coluna, "SELLOUT":"SELLOUT %s" % coluna})                       #Nomear coluna nova
    
#dfMeio2 = pd.merge(dfMeio2, dfDelta[["DATA","STORE","Vendedor"]] , how="left", on = ["DATA","STORE"])  
dfFinal2=dfMeio2.copy()


In [406]:
dfFinal2[dfFinal2.STORE==760]

,Loja,STORE,DATA,CERVEJA FRANZISKANER WEISSBIER 50 CL 5%,"CERVEJA CORONA 35,5CL (UN)",CERVEJA LEFFE BLONDE 33 CL CX24,Num_Produtos,Vendedor,STOCK CERVEJA FRANZISKANER WEISSBIER 50 CL 5%,SELLOUT CERVEJA FRANZISKANER WEISSBIER 50 CL 5%,"STOCK CERVEJA CORONA 35,5CL (UN)","SELLOUT CERVEJA CORONA 35,5CL (UN)",STOCK CERVEJA LEFFE BLONDE 33 CL CX24,SELLOUT CERVEJA LEFFE BLONDE 33 CL CX24
8,Pingo Doce Armação de Pêra - Torre,760,2023-08-20,0.0,1.0,1.0,2.0,Cátia Águas,20,0,57.0,0.0,33.0,0.0


---

> STK

> Balanço

In [407]:
for i in produtos:
    stk = "STOCK %s" %i
    #msa = "MSA %s" %i
    sellout = "SELLOUT %s" %i
    
    balance = "BALANCE %s" %i
    
    dfFinal2[balance] = dfFinal2[sellout] / dfFinal2[stk]
    
    

> Rotura

In [408]:
for i in produtos:
    rotura  = "ROTURA %s" % i
    stock  = "STOCK %s" % i

    
    dfFinal2[rotura] = np.where((dfFinal2[stock] > 0) & ~(dfFinal2[i].isna()), 0, 
                      np.where((dfFinal2[stock] <= 0) & ~(dfFinal2[i].isna()), 1, 
                      np.nan))


> Pré-Rotura

>Sinal

In [422]:
#dfFinal2["Sinal"]= pd.Series(dtype='int')

# Define the mappings of numbers to strings
mapping = {1: "Presente com Stock",
           2: "Ausente com Stock",
           3: "Presente sem Stock",
           4: "Ausente sem Stock"}

for i in produtos:
    sinal = "SINAL %s" % i
    stock  = "STOCK %s" % i
    ninjainfo = i
    dfFinal2[sinal] = np.where((dfFinal2[stock] > 0) & (dfFinal2[i] == 1), 1,
                     np.where((dfFinal2[stock] > 0) & (dfFinal2[i] == 0), 2,
                     np.where((dfFinal2[stock] <= 0) & (dfFinal2[i] == 1), 3,
                     np.where((dfFinal2[stock] <= 0) & (dfFinal2[i] == 0), 4,
                     np.nan))))


    # Map the numbers to the corresponding strings
    dfFinal2[sinal] = dfFinal2[sinal].map(mapping)
    dfFinal2[ninjainfo] = np.where(dfFinal2[stock].isna(), np.nan, dfFinal2[i])

# 1: Há Stock e Sinal Positivo Ninja "Verdadeiramente Correcto"
# 2: Há Stock e Sinal Negativo Ninja "Falso Positivo"
# 3: Não há Stock e Sinal Positivo Ninja "Falso Negativo"
# 4: Não há Stock e Sinal Negativo Ninja "Rejeição Correcta"

> Ciclos

> Adequação

- Dias para a rotura

In [423]:
for i in produtos:
    stock  = "STOCK %s" % i
    #msa = "MSA %s" %i
    sellout = "SELLOUT %s" % i
    dias_rotura = "Dias_para_Rotura %s" % i
    
    dfFinal2[dias_rotura] = dfFinal2[stock]/dfFinal2[sellout]

- Dias para a rotura de prateleira

---

# `2022`

#### dfFinal2_2022

## <center><font color=orange>Output DataFusion Wide

In [424]:
df2 = dfFinal2.copy()
#df2_2022 = dfFinal2_2022.copy()

[Escrever](#section5) <br>
[Visualizar](#section6)

---

<a id="section4"></a>
# <center>4. Gráficos (3)</center>
[Voltar ao índice](#section0)<br>

> - __produtos__ já definidos

---

- #  `dfFinal3`

In [425]:
dfFinal3 = df2.copy()

# Tabelas

# - <span style="color:green">Classic:</span>  Proporção de presença por produto

In [429]:
dfFinal3[produtos[2]].sum()

76.0

In [427]:
# Média tirando os missing values
df_Med = dfFinal3[produtos].mean(numeric_only=True)#.groupby('Altura_do_Dia').mean()

# Passar para dataframe
df_Classic = df_Med.to_frame()

# Mudanças para ficar legível
df_Classic = df_Classic.reset_index()
df_Classic = df_Classic.rename(columns={'index': 'Produto', 0: 'Frequência'})

In [428]:
df_Classic

,Produto,Frequência
0,CERVEJA FRANZISKANER WEISSBIER 50 CL 5%,0.903614
1,"CERVEJA CORONA 35,5CL (UN)",0.829268
2,CERVEJA LEFFE BLONDE 33 CL CX24,0.926829


- ###  <span style="color:BLUE">Com e sem Reposição</span>

>> <font color=blue>Com</font>

>> <font color=blue>Sem

---

# - <font color=green> Final</font>: Proporção de presença por loja

In [430]:
# com reposição
# dfFinal3["Num_ProdutosRep"] = np.nan
# dfFinal3.loc[dfFinal3['Repos'] == 1, 'Num_ProdutosRep'] = dfFinal3["Num_Produtos"]


# # sem reposição
# dfFinal3["Num_ProdutosNRep"] = np.nan
# dfFinal3.loc[dfFinal3['Repos'] == 0, 'Num_ProdutosNRep'] = dfFinal3["Num_Produtos"]

# Coluna para ter o número de produtos contratados pela marca
dfFinal3["Produtos_Contratados"] = dfFinal3[produtos].notna().sum(axis=1)

In [431]:
'''Fazer colunas de proporção de produtos presentes em loja, depois especificando para quando têm e não têm reposição'''

df_NumProd=dfFinal3[["Loja","Num_Produtos"]].copy()
df_NumProd["Prop_Produtos"] = (df_NumProd["Num_Produtos"] / dfFinal3["Produtos_Contratados"])*100
# df_NumProd["Prop_ProdutosRep"] = (df_NumProd["Num_ProdutosRep"] / dfFinal3["Produtos_Contratados"])*100
# df_NumProd["Prop_ProdutosNRep"] = (df_NumProd["Num_ProdutosNRep"] / dfFinal3["Produtos_Contratados"])*100

In [432]:
# Acrescentar à base original
dfFinal3["Prop_Produtos"] = df_NumProd["Prop_Produtos"]

---

# - <font color=green> Roturas</font>: Proporção de roturas por produto

In [433]:
df_RotMed = dfFinal3[["ROTURA %s" % produto for produto in produtos]].mean(numeric_only=True)

# Passar para dataframe
df_Rot = df_RotMed.to_frame()
df_Rot = df_Rot.reset_index()
df_Rot = df_Rot.rename(columns={'index': 'Produto', 0: 'Frequencia'})
df_Rot["Frequencia"] = df_Rot["Frequencia"]*100

## <center><font color=orange>Output DataFusion Wide

In [434]:
df2_2 = dfFinal3

#df_Classic
#df_Classic_ComRepo
#df_Classic_SemRepo
#df_Rot
#df_NumProd

[Visualizar](#section6)

---

<a id="section5"></a>
# <center><u>Escrever</u></center>
[Voltar ao índice](#section0)

### <center><font color=green>DataCleaning</font></center>

In [435]:
escrever_txt(df_produtos, "produtos")
escrever_txt(df_lojas, "lojas")

### <center><font color=green>DataFusion Long</font></center>

In [436]:
escrever_excel(df1, "Long_DataFusion")

### <center><font color=green>DataFusion Wide</font></center>

In [437]:
#escrever_excel(df2_1, "Wide_DataFusion_Inc")

escrever_excel(df2_2, "Wide_DataFusion2")
#escrever_excel(df2_2022, "Wide_DataFusion2022")

### <center><font color=green>Gráficos</font></center>


- Classic

In [438]:
# Normal
escrever_excel(df_Classic, "G_Classic")

# Com Reposição
# escrever_excel(df_Classic_ComRepo, "G_Classic_Reposto")

# # Sem Reposição
# escrever_excel(df_Classic_SemRepo, "G_Classic_SemReposto")

- Roturas

In [439]:
produtos

['CERVEJA FRANZISKANER WEISSBIER 50 CL 5%',
 'CERVEJA CORONA 35,5CL (UN)',
 'CERVEJA LEFFE BLONDE 33 CL CX24']

In [440]:
escrever_excel(df_Rot, "G_Rotura")

- Número de produtos

In [455]:
escrever_excel(df_NumProd, "G_NumProdutos")

<a id="section6"></a>
# <center><u>Visualizar</u></center>
[Voltar ao índice](#section0)

In [445]:
df_NumProd

,Loja,Num_Produtos,Prop_Produtos
0,Pingo Doce Albufeira - Bela Vista,2.0,66.666667
1,Pingo Doce Albufeira - Quintata da Correeira,3.0,100.000000
2,Pingo Doce Alcântara,3.0,100.000000
3,Pingo Doce Algés,3.0,100.000000
4,Pingo Doce Almada,3.0,100.000000
...,...,...,...
80,Pingo Doce Torres da Luz,3.0,100.000000
81,Pingo Doce Valadares - Rio do Paço,3.0,100.000000
82,Pingo Doce Vale do Lobo,3.0,100.000000
83,Pingo Doce Venda Nova - Elias Garcia,3.0,100.000000


In [451]:
df_NumProd.loc[df_NumProd["Prop_Produtos"] > 100, "Prop_Produtos"] = 100.0

In [381]:
len(df_NumProd[df_NumProd.Prop_Produtos==100])/85

0.6941176470588235

In [368]:
[f"{index}: {column}" for index, column in enumerate(df2_2)]

['0: Loja',
 '1: STORE',
 '2: DATA',
 '3: CERVEJA FRANZISKANER WEISSBIER 50 CL 5%',
 '4: CERVEJA CORONA 35,5CL (UN)',
 '5: CERVEJA LEFFE BLONDE 33 CL CX24',
 '6: Num_Produtos',
 '7: Vendedor',
 '8: STOCK CERVEJA FRANZISKANER WEISSBIER 50 CL 5%',
 '9: SELLOUT CERVEJA FRANZISKANER WEISSBIER 50 CL 5%',
 '10: STOCK CERVEJA CORONA 35,5CL (UN)',
 '11: SELLOUT CERVEJA CORONA 35,5CL (UN)',
 '12: STOCK CERVEJA LEFFE BLONDE 33 CL CX24',
 '13: SELLOUT CERVEJA LEFFE BLONDE 33 CL CX24',
 '14: BALANCE CERVEJA FRANZISKANER WEISSBIER 50 CL 5%',
 '15: BALANCE CERVEJA CORONA 35,5CL (UN)',
 '16: BALANCE CERVEJA LEFFE BLONDE 33 CL CX24',
 '17: ROTURA CERVEJA FRANZISKANER WEISSBIER 50 CL 5%',
 '18: ROTURA CERVEJA CORONA 35,5CL (UN)',
 '19: ROTURA CERVEJA LEFFE BLONDE 33 CL CX24',
 '20: SINAL CERVEJA FRANZISKANER WEISSBIER 50 CL 5%',
 '21: SINAL CERVEJA CORONA 35,5CL (UN)',
 '22: SINAL CERVEJA LEFFE BLONDE 33 CL CX24',
 '23: Dias_para_Rotura CERVEJA FRANZISKANER WEISSBIER 50 CL 5%',
 '24: Dias_para_Rotura C

- # `Long_DataFusion`

> Filtrar produto

In [ ]:
@interact
def filtrar_produto(produto = list(df1.DESC_ARTIGO.unique())):
    
    return df1[df1.DESC_ARTIGO == produto].head() # Alterável

> Filtrar loja

In [ ]:
@interact
def filtrar_loja(loja = list(df1.Loja.unique())):
    
    return df1[df1.Loja == loja].head() # Alterável

- # `Wide_DataFusion`

In [ ]:
import plotly.graph_objects as go

import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/violin_data.csv")


pointpos_male = [-0.9,-1.1,-0.6,-0.3]
pointpos_female = [0.45,0.55,1,0.4]
show_legend = [True,False,False,False]

fig = go.Figure()

for i in range(0,len(pd.unique(df['SELLOUT']))):
    fig.add_trace(go.Violin(x=df['SELLOUT'][(df['sex'] == 'Male') &
                                        (df['SELLOUT'] == pd.unique(df['SELLOUT'])[i])],
                            y=df['total_bill'][(df['sex'] == 'Male')&
                                               (df['SELLOUT'] == pd.unique(df['SELLOUT'])[i])],
                            legendgroup='M', scalegroup='M', name='M',
                            side='negative',
                            pointpos=pointpos_male[i], # where to position points
                            line_color='lightseagreen',
                            showlegend=show_legend[i])
             )
    fig.add_trace(go.Violin(x=df['SELLOUT'][(df['sex'] == 'Female') &
                                        (df['SELLOUT'] == pd.unique(df['SELLOUT'])[i])],
                            y=df['total_bill'][(df['sex'] == 'Female')&
                                               (df['SELLOUT'] == pd.unique(df['SELLOUT'])[i])],
                            legendgroup='F', scalegroup='F', name='F',
                            side='positive',
                            pointpos=pointpos_female[i],
                            line_color='mediumpurple',
                            showlegend=show_legend[i])
             )

# update characteristics shared by all traces
fig.update_traces(meanline_visible=True,
                  points='all', # show all points
                  jitter=0.05,  # add some jitter on points for better visibility
                  scalemode='count') #scale violin plot area with total count
fig.update_layout(
    title_text="Total bill distribution<br><i>scaled by number of bills per gender",
    violingap=0, violingroupgap=0, violinmode='overlay')
fig.show()

[Voltar ao índice](#section0)